In [1]:
import tensorflow as tf
import keras

from keras.preprocessing import sequence
from keras.models import Model
from keras.datasets import imdb
from keras.layers import Input, Embedding
from keras.layers import Dense, Activation, LSTM, Bidirectional

Using TensorFlow backend.


In [0]:
# set parameters:
max_features = 5000 # Put it up to 10k?
maxlen = 400
batch_size = 32
embedding_dims = 50
lstm_units = 100 # play with this
epochs = 2

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [4]:
# Network Architecture
var_input = Input(shape=(400,))
emb_layer = Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen)(var_input)
lstm_layer = Bidirectional(LSTM(units=lstm_units, activation='tanh', recurrent_activation='sigmoid', 
                  dropout=0.0, recurrent_dropout=0.0))(emb_layer)
"""how many units?
play w dropouts
[avoid vanishing gradient w TanH: second derivative can sustain for a long range before going to zero.]
try diff activations"""

output = Dense(1, activation='sigmoid')(lstm_layer)

model = Model(inputs=var_input, outputs=output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# Train
model.fit(x_train, y_train,
          batch_size=batch_size,
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)




Train on 25000 samples, validate on 25000 samples
Epoch 1/2





25000/25000 [==============================] - 1358s 54ms/step - loss: 0.5463 - acc: 0.7341 - val_loss: 0.4416 - val_acc: 0.7979
Epoch 2/2
25000/25000 [==============================] - 379s 15ms/step
Test score: 0.3421998379230499
Test accuracy: 0.85744
